Add training py scripts to path

In [1]:
import sys
import os

path_prefix = "/home/kanthoulis/spider/"

training_scripts_dir = path_prefix + "training"
transforms_dir = path_prefix + "transforms"
image_dir = path_prefix + "image"


sys.path.append(training_scripts_dir)
sys.path.append(transforms_dir)
sys.path.append(image_dir)


Dependencies

In [2]:
#UNet 
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.amp import autocast, GradScaler
import json 
import numpy as np
import re 
import os
import pathlib
from natsort import natsorted
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

#model imports 
import unet

import dataset_torch_numpy as dataset
import dataset_torch_numpy_aug as dataset_aug
import metric
#import epoch as ep #ep not to conlfict with var name in loop



/home/kanthoulis/jupyter-venv/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.23). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
print(torch.cuda.device_count())  # Ensure you have more than 1 GPU
print(torch.cuda.get_device_name(1))  # Name of the 2nd GPU (index 1)

"""
#Set GPU/Cuda Device to run model on
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

"""
#use 2nd gpu 
device = torch.device("cuda:1")
torch.cuda.set_device(1)
print(f"Using {device} device")
torch.manual_seed(46)


2
NVIDIA GeForce RTX 4090
Using cuda:1 device


Load tensor data 

In [4]:
json_path = "/home/kanthoulis/spider/tensor_data/tensor_data.json" 

#Load tensor parameters from .json
with open(json_path, 'r') as file:
    data = json.load(file)

#Assign each value to a variable
row_max = data["row_max"]
col_max = data["col_max"]
image_tensor_min = data["image_tensor_min"]
image_tensor_max = data["image_tensor_max"]
label_tensor_min = data["label_tensor_min"]
label_tensor_max = data["label_tensor_max"]
masks_no = data["masks_no"]
masks_array = data["masks_array"]

print(masks_no)

20


Data Load

In [5]:
path_prefix = "/home/kanthoulis/spider/dataset/"

train_img_slice_dir = pathlib.Path(path_prefix + "train_image_numpy") #TODO SET PATH IN JHUB
train_label_slice_dir = pathlib.Path(path_prefix + "train_label_numpy") #TODO SET PATH IN JHUB 
test_img_slice_dir = pathlib.Path(path_prefix + "test_image_numpy") #TODO SET PATH IN JHUB
test_label_slice_dir= pathlib.Path(path_prefix + "test_label_numpy") 

#Create lists of filenames in directories (str)
image_path = train_img_slice_dir
label_path = test_label_slice_dir

image_dir_list = os.listdir(image_path)
label_dir_list = os.listdir(label_path)

#Sort lists
image_dir_list = natsorted(image_dir_list)
label_dir_list = natsorted(label_dir_list)

print("train dataset len",image_dir_list.__len__())
print("test dataset len",label_dir_list.__len__())

train dataset len 11484
test dataset len 3197


Dice Loss

In [6]:
import torch
import torch.nn.functional as F

def dice_loss(pred, target, num_classes=20, smooth=1, epsilon=1e-6):
    # Ensure target shape is [batch_size, H, W]
    target = target.squeeze(1)  # Remove unnecessary channel dimension if present

    # Apply softmax to get probabilities from logits
    pred = F.softmax(pred, dim=1)  # Shape: [batch_size, num_classes, H, W]

    # Convert target to one-hot encoding
    target = F.one_hot(target.long(), num_classes=num_classes)  # Shape: [batch_size, H, W, num_classes]
    target = target.permute(0, 3, 1, 2).float()  # Shape: [batch_size, num_classes, H, W]

    # Compute intersection and union
    intersection = (pred * target).sum(dim=(2, 3))  # Shape: [batch_size, num_classes]
    union = pred.sum(dim=(2, 3)) + target.sum(dim=(2, 3))  # Shape: [batch_size, num_classes]

    # Prevent division by zero by adding epsilon
    union = union + epsilon

    # Compute per-class Dice score
    dice = (2. * intersection + smooth) / (union + smooth)  # Shape: [batch_size, num_classes]

    # Define class weights (0.1 for background, 1.0 for others)
    class_weights = torch.ones(num_classes, device=pred.device)
    class_weights[0] = 0.1  # Background class weight

    # Compute weighted Dice loss
    weighted_dice = class_weights * (1 - dice)  # Shape: [batch_size, num_classes]
    loss = weighted_dice.mean()  # Average over batch and classes

    return loss


In [7]:
#CrossEntropyLoss

class CombinedLoss(torch.nn.Module):
    def __init__(self, ce_weight=0.5, dice_weight=0.5):
        super(CombinedLoss, self).__init__()
        self.ce_weight = ce_weight
        self.dice_weight = dice_weight
        self.ce_loss = torch.nn.CrossEntropyLoss()
        weights = torch.cat([torch.tensor([0.1]), torch.ones(19)])
        self.ce_loss = torch.nn.CrossEntropyLoss()

    def forward(self, outputs, targets):
        # Cross Entropy Loss expects class probabilities and the raw target labels (not one-hot)
        #ce_loss = self.ce_loss(outputs, torch.argmax(targets, dim=1))
        ce_loss = self.ce_loss(outputs, targets.squeeze(1).long()) 
        
        # Dice Loss expects one-hot encoded targets and softmax probabilities
        dice_loss_val = dice_loss(outputs, targets)
        
        # Weighted sum of the two losses
        return self.ce_weight * ce_loss + self.dice_weight * dice_loss_val


In [8]:
#BCEWithLogits
"""
class CombinedLoss(nn.Module):
    def __init__(self, ce_weight=0.5, dice_weight=0.5):
        super(CombinedLoss, self).__init__()
        self.ce_weight = ce_weight
        self.dice_weight = dice_weight
        self.ce_loss = nn.BCEWithLogitsLoss()

    def forward(self, outputs, targets):
    
        #Compute the combined loss.
       # Args:
        #    outputs: Logits from the model (shape: B x C x H x W).
        #    targets: Ground truth labels (shape: B x C x H x W, binary).
        #Returns:
          #  Combined loss value.
         
        # Compute BCEWithLogitsLoss
        ce_loss = self.ce_loss(outputs, targets)
        
        # Compute Dice Loss
        dice_loss_val = dice_loss(outputs, targets)
        
        # Weighted sum of the two losses
        return self.ce_weight * ce_loss + self.dice_weight * dice_loss_val
"""


'\nclass CombinedLoss(nn.Module):\n    def __init__(self, ce_weight=0.5, dice_weight=0.5):\n        super(CombinedLoss, self).__init__()\n        self.ce_weight = ce_weight\n        self.dice_weight = dice_weight\n        self.ce_loss = nn.BCEWithLogitsLoss()\n\n    def forward(self, outputs, targets):\n    \n        #Compute the combined loss.\n       # Args:\n        #    outputs: Logits from the model (shape: B x C x H x W).\n        #    targets: Ground truth labels (shape: B x C x H x W, binary).\n        #Returns:\n          #  Combined loss value.\n         \n        # Compute BCEWithLogitsLoss\n        ce_loss = self.ce_loss(outputs, targets)\n        \n        # Compute Dice Loss\n        dice_loss_val = dice_loss(outputs, targets)\n        \n        # Weighted sum of the two losses\n        return self.ce_weight * ce_loss + self.dice_weight * dice_loss_val\n'

Unet Torch - Model Parameters 

In [9]:
#Input Output channels
input_channels = 1 #Hounsfield scale, do not modify
output_channels = masks_no - 1 #-1 to exclude backround value 0, do not modify\

#Model Hyperparams 
depth = 5
#start_filts = 32
start_filts = 64

up_mode = 'upsample'
#lr = 0.001 #adjust according to batch size 
lr = 0.0001
batchsize = 8
#batchsize = 8 
#loss_func = nn.BCEWithLogitsLoss() 
#loss_func = nn.CrossEntropyLoss(ignore_index = 0)

one_hot_labels = False #false for ce true for bce

#weights = torch.cat([torch.tensor([0.1]), torch.ones(19)])
#loss_func  = torch.nn.CrossEntropyLoss(weight = weights)

loss_func = CombinedLoss()

loss_func.to(device)

# Assigning the dice_loss function as the loss function (without calling it)
"""
from functools import partial

loss_func = partial(dice_loss,num_classes = masks_no)
"""



'\nfrom functools import partial\n\nloss_func = partial(dice_loss,num_classes = masks_no)\n'

Unet Torch - Model & Optimizer Init 

In [10]:
model = unet.UNet(in_channels= input_channels,num_classes=masks_no, depth= depth, start_filts=start_filts, up_mode=up_mode) 

model.to(device)
model.to(torch.float32)  

optim = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.001) #AdamW
#optim = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.0001) #AdamW

--

In [11]:
# Initialize the GradScaler
#scaler = GradScaler() 


In [12]:
#FIRST TRAINING SESSION ONLY
epoch_no = 0 

In [13]:
"""
#NOTE: a simple script to get the file w the highest idx in the directory wouldn't be hard to iterate but for now
    #just focusing on training the model, maybe work on it further down the line 

#TODO also load scheduler when resuming training (when used)

#last epoch trained path
path = "/home/kanthoulis/spider/models/spider_seg_19" #last epoch pre scheduler


checkpoint= torch.load(path)
print(checkpoint.keys())
#optim = torch.optim.Adam(model.parameters(), lr=lr)
optim = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.001)


def extract_number_from_path(path):
    match = re.search(r'(\d+)$', path)
    return int(match.group(1)) if match else None

epoch_no = extract_number_from_path(path) + 1 #number for plotting in tb
print(epoch_no)

model.load_state_dict(checkpoint['model_dict'])
optim.load_state_dict(checkpoint['optimizer_dict'])
#scaler.load_state_dict(checkpoint['scaler_state_dict'])

model.to(device)
model.to(torch.float32)
"""


<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_1860126/2699515293.py:1: SyntaxWarning: invalid escape sequence '\d'
  """


'\n#NOTE: a simple script to get the file w the highest idx in the directory wouldn\'t be hard to iterate but for now\n    #just focusing on training the model, maybe work on it further down the line \n\n#TODO also load scheduler when resuming training (when used)\n\n#last epoch trained path\npath = "/home/kanthoulis/spider/models/spider_seg_19" #last epoch pre scheduler\n\n\ncheckpoint= torch.load(path)\nprint(checkpoint.keys())\n#optim = torch.optim.Adam(model.parameters(), lr=lr)\noptim = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.001)\n\n\ndef extract_number_from_path(path):\n    match = re.search(r\'(\\d+)$\', path)\n    return int(match.group(1)) if match else None\n\nepoch_no = extract_number_from_path(path) + 1 #number for plotting in tb\nprint(epoch_no)\n\nmodel.load_state_dict(checkpoint[\'model_dict\'])\noptim.load_state_dict(checkpoint[\'optimizer_dict\'])\n#scaler.load_state_dict(checkpoint[\'scaler_state_dict\'])\n\nmodel.to(device)\nmodel.to(torch.float3

UNet Torch - Datasets & Dataloaders

In [14]:
#Datasets 
train_set = dataset.SpiderDatasetNumpy(train_label_slice_dir, train_img_slice_dir, one_hot_labels = one_hot_labels, augmentation = True)
train_eval_set = dataset.SpiderDatasetNumpy(train_label_slice_dir, train_img_slice_dir, one_hot_labels = one_hot_labels, augmentation = False)
test_set = dataset.SpiderDatasetNumpy(test_label_slice_dir, test_img_slice_dir, one_hot_labels = one_hot_labels, augmentation = False)

#Dataloaders
train_dataloader = DataLoader(train_set, batch_size = batchsize, shuffle=True)
train_eval_dataloader = DataLoader(train_eval_set, batch_size = 1, shuffle=True) #for more accurate metrics
test_dataloader = DataLoader(test_set, batch_size = 1, shuffle=True) #for more accurate metrics

print(len(train_eval_dataloader))
print(len(test_dataloader))

11484
3197


Cosine Annealing

In [15]:
from torch.optim.lr_scheduler import CosineAnnealingLR

# Scheduler setup
T_max = len(train_dataloader)  # One full cycle is one epoch
eta_min = 1e-6                 # Minimum learning rate

scheduler = CosineAnnealingLR(
    optimizer=optim,
    T_max=T_max,
    eta_min=eta_min
)

#scheduler.load_state_dict(checkpoint['scheduler_dict']) # UNCOMMENT AFTER SCHEDULER IS INIT


LR Scheduler CosineAnnealingWarmRestarts

In [16]:
"""
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

# Scheduler setup
T_0 = len(train_dataloader)  # One epoch per cycle
T_mult = 2                   # Each subsequent cycle is 2x longer than the previous
eta_min = 1e-6               # Minimum learning rate

scheduler = CosineAnnealingWarmRestarts(
    optimizer=optim,
    T_0=T_0,
    T_mult=T_mult,
    eta_min=eta_min
)

scheduler.load_state_dict(checkpoint['scheduler_dict']) #UNCOMMENT AFTER SCHEDULER IS INIT
"""

"\nfrom torch.optim.lr_scheduler import CosineAnnealingLR\nfrom torch.optim.lr_scheduler import CosineAnnealingWarmRestarts\n\n# Scheduler setup\nT_0 = len(train_dataloader)  # One epoch per cycle\nT_mult = 2                   # Each subsequent cycle is 2x longer than the previous\neta_min = 1e-6               # Minimum learning rate\n\nscheduler = CosineAnnealingWarmRestarts(\n    optimizer=optim,\n    T_0=T_0,\n    T_mult=T_mult,\n    eta_min=eta_min\n)\n\nscheduler.load_state_dict(checkpoint['scheduler_dict']) #UNCOMMENT AFTER SCHEDULER IS INIT\n"

LR Scheduler reduceonPlateau

In [17]:
"""

# Define the ReduceLROnPlateau scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, 
    mode='min', 
    factor=0.5, 
    patience=3, 
    verbose=False, 
    min_lr=1e-6)



"""
#scheduler.load_state_dict(checkpoint['scheduler_dict'])

"\n\n# Define the ReduceLROnPlateau scheduler\nscheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(\n    optim, \n    mode='min', \n    factor=0.5, \n    patience=3, \n    verbose=False, \n    min_lr=1e-6)\n\n\n\n"

Batch Accumulation

In [18]:
accumulation_steps  = 4 #4*16 = 64
#for batch size 8 4*8 = 32

Set no. of epochs to train for training session

In [19]:
epochs = 60

In [20]:
from torchmetrics import Accuracy, Precision, Recall, Dice, F1Score

"""
accuracy_metric = Accuracy(task='multiclass', num_classes=masks_no, average = 'macro', ignore_index = 0).to(device)
precision_metric = Precision(task='multiclass', num_classes=masks_no, average = 'macro', ignore_index = 0).to(device)
recall_metric = Recall(task='multiclass', num_classes=masks_no, average = 'macro', ignore_index = 0).to(device)
dice_metric = F1Score(task='multiclass', num_classes=masks_no, average = 'macro', ignore_index = 0).to(device)
"""

accuracy_metric = Accuracy(task='multiclass', num_classes=masks_no, average = 'macro').to(device)
precision_metric = Precision(task='multiclass', num_classes=masks_no, average = 'macro').to(device)
recall_metric = Recall(task='multiclass', num_classes=masks_no, average = 'macro').to(device)
dice_metric = F1Score(task='multiclass', num_classes=masks_no, average = 'macro').to(device)

accuracy_metric.reset()
dice_metric.reset()
precision_metric.reset()
recall_metric.reset()


Also save valid metrics locally as csv

In [21]:
import os
import csv

# Define the directory where the file will be stored
csv_directory = '/home/kanthoulis/spider/runs/'  # Replace with the correct path

# Define the path to the CSV file
csv_filename = os.path.join(csv_directory, 'valid_metrics.csv') #CHANGE ACCORDINGLY

# Define the header for the CSV file
header = [
    'epoch',  # Epoch
    'avg_tloss', 'avg_vloss',  # Training/Validation Loss
    'avg_taccu', 'avg_vaccu',  # Training/Validation Accuracy
    'avg_tdice', 'avg_vdice',  # Training/Validation Dice
    'avg_tprec', 'avg_vprec',  # Training/Validation Precision
    'avg_trecall', 'avg_vrecall'  # Training/Validation Recall
]

def save_metrics_to_csv(epoch, metrics):
    # Check if the file exists
    file_exists = os.path.isfile(csv_filename)
    data = []

    # Read existing data if file exists
    if file_exists:
        with open(csv_filename, mode='r', newline='') as file:
            reader = csv.reader(file)
            data = list(reader)

    # Check if the header exists, otherwise add it
    if not data or data[0] != header:
        data = [header]

    # Convert epoch to string for comparison
    epoch_str = str(epoch)
    row_found = False

    # Update row if epoch exists
    for i in range(1, len(data)):  # Skip header row
        if data[i][0] == epoch_str:  
            data[i] = [epoch_str] + metrics  # Overwrite existing row
            row_found = True
            break

    # Append new row if epoch was not found
    if not row_found:
        data.append([epoch_str] + metrics)

    # Write the updated data back to the CSV file
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)


Train Loop

In [22]:
from tqdm import tqdm
import torch
from torch.cuda.amp import autocast
from torchmetrics import Accuracy, Precision, Recall, Dice

def evaluate_model(dataloader, model, device, loss_func, desc):
    # Initialize torchmetrics instances
    # Initialize running metrics
    running_loss = 0.0
    
    model.eval()  # Ensure model is in evaluation mode

    with torch.no_grad():  # Disable gradient computation for evaluation
        for _, data in enumerate(tqdm(dataloader, desc=desc)):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
                    #with autocast(device.type): 
            with autocast():# AMP handles precision here
                outputs = model(inputs)
                #loss = loss_func(outputs, labels.squeeze(1).long()) #ce only
                loss = loss_func(outputs, labels) #dice and combloss

            # Convert one-hot encoded labels to class indices
            labels_preds = labels.squeeze(1).long()  # Shape: (16, 576, 576)
            preds = torch.argmax(outputs, dim=1)        # Shape: (16, 576, 576)
                        # Overall Metrics

            preds = preds.to(device)
            labels_preds = labels_preds.to(device)

            running_loss += loss.item()
            accuracy_metric.update(preds, labels_preds)
            dice_metric.update(preds, labels_preds)
            precision_metric.update(preds, labels_preds)
            recall_metric.update(preds, labels_preds)

           
    # Calculate averages
    avg_loss = running_loss / len(dataloader)
    avg_accu = accuracy_metric.compute()
    avg_dice = dice_metric.compute()
    avg_prec = precision_metric.compute()
    avg_recall = recall_metric.compute()


    return {
        "avg_loss": avg_loss,
        "avg_accu": avg_accu,
        "avg_dice": avg_dice,
        "avg_prec": avg_prec,
        "avg_recall": avg_recall
    }


In [23]:
#timestamp = datetime.now().strftime('%Y%m%d_%H')
#writer = SummaryWriter('runs/spider_seg_unet_epochs={}_lr={}_batchsize={}_loss=BCEWithLogits_startfilts={}_upmode={}'.format(epochs,lr, batchsize,start_filts,up_mode))
#tb_writer = SummaryWriter('/home/kanthoulis/spider/runs_19onehot/spider_seg') #CHANE PER TRAINING SESSION AVOID TB OVERWRITES
epoch_number = epoch_no #set to no of last epoch completed to have consistent tb plots 

best_vloss = 1_000_000.

for epoch in range(epochs):
    print('EPOCH {}:'.format(epoch_number + 1))

    #----------------------Train Loop---------------------------------
    model.train(True) #set model to train
    #Instantiate training metrics per epoch
    running_loss = 0.
    last_loss = 0.

    accuracy_metric.reset()
    dice_metric.reset()
    precision_metric.reset()
    recall_metric.reset()
        
    
    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting

    progress_bar = tqdm(train_dataloader, desc="Training", leave=True)
    for i, data in enumerate(progress_bar):


        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero gradients at the start of each accumulation step
        if i % accumulation_steps == 0:
            optim.zero_grad()

        """
        # Use AMP for mixed precision training
        with autocast():
            outputs = model(inputs)
            loss = loss_func(outputs, labels) #dice and combloss
            #loss = loss_func(outputs, labels.squeeze(1).long()) #ce only 

            # Check for NaNs/Infs in outputs and labels
            if torch.any(torch.isnan(outputs)) or torch.any(torch.isinf(outputs)):
                print(f"NaNs or Infs detected in outputs at batch {i}")
            if torch.any(torch.isnan(labels)) or torch.any(torch.isinf(labels)):
                print(f"NaNs or Infs detected in labels at batch {i}")

        progress_bar.set_postfix(loss=f"{loss.item():.4f}")
        running_loss += loss.item()
        # Scale the loss for accumulation
        loss = loss / accumulation_steps  # Prevent gradient explosion
        
        # Backpropagate with AMP
        scaler.scale(loss).backward()
        
        # Update weights only after accumulation steps
        if (i + 1) % accumulation_steps == 0:
            scaler.unscale_(optim)  # Unscale before clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Clip gradients
            scaler.step(optim)  # Perform optimizer step
            scaler.update()  # Update the scaler
            scheduler.step()  # Adjust learning rate
        """

        # Forward pass
        outputs = model(inputs)
        loss = loss_func(outputs, labels)  # dice and combloss
        # loss = loss_func(outputs, labels.squeeze(1).long())  # ce only 

        # Check for NaNs/Infs in outputs and labels
        if torch.any(torch.isnan(outputs)) or torch.any(torch.isinf(outputs)):
            print(f"NaNs or Infs detected in outputs at batch {i}")
        if torch.any(torch.isnan(labels)) or torch.any(torch.isinf(labels)):
            print(f"NaNs or Infs detected in labels at batch {i}")

        progress_bar.set_postfix(loss=f"{loss.item():.4f}")
        running_loss += loss.item()
        # Scale the loss for accumulation
        loss = loss / accumulation_steps  # Prevent gradient explosion
        
        # Backpropagation without AMP
        loss.backward()

        # Update weights only after accumulation steps
        if (i + 1) % accumulation_steps == 0:
            # Clip gradients to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Clip gradients
            optim.step()  # Perform optimizer step
            scheduler.step()  # Adjust learning rate


        preds = torch.argmax(outputs, dim=1).long()  # Shape: (16, 576, 576)
        labels_preds = labels.squeeze(1).long()
        # Convert one-hot encoded labels to class indices
        #labels_preds = torch.argmax(labels, dim=1)  # Shape: (16, 576, 576)

        
        # Flatten for metric computation
        preds_flat = preds.view(-1).long()    # Shape: (16 * 576 * 576,)
        labels_flat = labels_preds.view(-1).long()  # Shape: (16 * 576 * 576,)
        
        # Ensure matching shapes
        assert preds_flat.shape == labels_flat.shape, f"Mismatch: {preds_flat.shape} vs {labels_flat.shape}"

        preds = preds.to(device)
        labels_preds = labels_preds.to(device)

        # General Metrics
        accuracy_metric.update(preds, labels_preds)
        dice_metric.update(preds, labels_preds)
        precision_metric.update(preds, labels_preds)
        recall_metric.update(preds, labels_preds)

        # Gather data and report
        #Intra training
        

    #-----------------Eval loop------------------------------------
    #print("avg loss in epoch", avg_loss)
    avg_loss = running_loss / len(train_dataloader)
    avg_accu = accuracy_metric.compute()
    avg_dice = dice_metric.compute()
    avg_prec = precision_metric.compute()
    avg_recall = recall_metric.compute()

    accuracy_metric.reset()
    dice_metric.reset()
    precision_metric.reset()
    recall_metric.reset()
    
    running_vloss = 0.0

    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.

    #if(epoch_number + 1) % 5 == 0:
    if(True):
        model.eval()
    
        # Disable gradient computation and reduce memory consumption.
        with torch.no_grad():
    
            test_metrics = evaluate_model(
                dataloader=test_dataloader,
                model=model,
                device=device,
                loss_func=loss_func,
                desc="Evaluating Test Set"
            )
    

            avg_vloss = test_metrics["avg_loss"]
            avg_vaccu = test_metrics["avg_accu"]
            avg_vdice = test_metrics["avg_dice"]
            avg_vprec = test_metrics["avg_prec"]
            avg_vrecall = test_metrics["avg_recall"]
            
            # Scheduler
            # scheduler.step(avg_vloss)
            
            # Logging
            print('Loss: Train {} Valid {}'.format(avg_loss, avg_vloss))
            print('Accuracy: Train {} Valid {}'.format(avg_accu, avg_vaccu))
            print('Dice: Train {} Valid {}'.format(avg_dice, avg_vdice))
            print('Precision: Train {} Valid {}'.format(avg_prec, avg_vprec))
            print('Recall: Train {} Valid {}'.format(avg_recall, avg_vrecall))
            
            

            # Replace with your calculated metrics
            metrics_for_csv = [
                avg_loss, avg_vloss,    # Training/Validation Loss
                avg_accu, avg_vaccu,    # Training/Validation Accuracy
                avg_dice, avg_vdice,    # Training/Validation Dice
                avg_prec, avg_vprec,    # Training/Validation Precision
                avg_recall, avg_vrecall,  # Training/Validation Recall
            ]


            save_metrics_to_csv(epoch_number + 1, metrics_for_csv)
                    
    
    #Change path to save model accordingly     
    model_path = '/home/kanthoulis/spider/models/spider_seg_{}'.format(epoch_number) #CHANGE ACCORDINGLY

    
    torch.save({
        'model_dict': model.state_dict(),
        'optimizer_dict': optim.state_dict(),
        'scheduler_dict': scheduler.state_dict(),
        #'scaler_state_dict': scaler.state_dict() #amp
    }, model_path)
    
    epoch_number += 1

EPOCH 1:


Training: 100%|██████████| 1436/1436 [12:39<00:00,  1.89it/s, loss=0.5726]
/tmp/ipykernel_1860126/4201957452.py:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():# AMP handles precision here
Evaluating Test Set: 100%|██████████| 3197/3197 [01:25<00:00, 37.34it/s]


Loss: Train 0.4672582568107873 Valid 0.3355000061785565
Accuracy: Train 0.07953856885433197 Valid 0.21643495559692383
Dice: Train 0.09814274311065674 Valid 0.196956068277359
Precision: Train 0.20715293288230896 Valid 0.23202691972255707
Recall: Train 0.07953856885433197 Valid 0.21643495559692383
EPOCH 2:


Evaluating Test Set: 100%|██████████| 3197/3197 [01:24<00:00, 37.65it/s]


Loss: Train 0.3024936845022822 Valid 0.23467866260408252
Accuracy: Train 0.2865317463874817 Valid 0.3527800440788269
Dice: Train 0.31793785095214844 Valid 0.3780660033226013
Precision: Train 0.36771342158317566 Valid 0.4304944574832916
Recall: Train 0.2865317463874817 Valid 0.3527800440788269
EPOCH 3:


...

Loss: Train 0.06883535757764635 Valid 0.1081553628757004
Accuracy: Train 0.8317500948905945 Valid 0.7068795561790466
Dice: Train 0.8445819020271301 Valid 0.7052481174468994
Precision: Train 0.8715482950210571 Valid 0.7044336795806885
Recall: Train 0.8317500948905945 Valid 0.7068795561790466
EPOCH 45:


Evaluating Test Set: 100%|██████████| 3197/3197 [01:25<00:00, 37.54it/s]


Loss: Train 0.06844220857569185 Valid 0.10934338684995233
Accuracy: Train 0.8317460417747498 Valid 0.7043145895004272
Dice: Train 0.8454004526138306 Valid 0.6979129910469055
Precision: Train 0.8739330172538757 Valid 0.6933956742286682
Recall: Train 0.8317460417747498 Valid 0.7043145895004272
EPOCH 46:


Training:  38%|███▊      | 540/1436 [04:43<07:50,  1.90it/s, loss=0.0725]


KeyboardInterrupt: 

# SHUT DOWN KERNELS AFTER TRAINING TO FREE UP MEMORY